In [ ]:
import json  #to be needed to read and load the message.json files that FB provides - https://docs.python.org/3/library/json.html
import os    #import os library to access folders in windows - https://docs.python.org/3/library/os.html
import ftfy  #library to fix the Greek characters mojibake error
import pandas as pd
from datetime import datetime as dt

In [ ]:
os.getcwd()

In [ ]:
#define a fix_text internal function
def mojibake(string):
    #return string  #just to kill the function and do nothing
    return ftfy.fix_text(string)

In [ ]:
# mojibake('ΟΞµ Ο€Ξ±ΞΉΞ΄ΞΉΞ¬ ΞµΞ³Ο Ξ¬Ξ»Ξ»ΞΏΟ… Ξ­ΟƒΟ„ΞµΞΉΞ»Ξ± Ο„ΞΏΟ…Ο‚ ΞµΞ―Ο€Ξ± π¤£')

In [ ]:
# download your Messenger data from Facebook
# set the path to the folder 

path_to_source_folder = 'D:/Kalimera/2023/downloaded_on_08.Jun.2023'

In [ ]:
#Create the Headers for the all 3 Dataset files

headers_conversation_ds = [
    'Key_C',
    'Participants',
    'Title',
    'Is_still_participant',
    'no_of_participants'
]

headers_messages_ds = [
    'Key_M',
    'Conversation',
    'Sender_name',
    'Timestamp_ms',
    'Message_text',
    'Message_type',
    'Call_duration',
    'Web_link',
    'Is_unsent'
]

headers_reactions_ds = [
    'Key_R',
    'Message',
    'Reaction',
    'Actor'    
]

headers_dict = {
    'conversations' : headers_conversation_ds,
    'messages' :headers_messages_ds,
    'reactions' : headers_reactions_ds
}

headers_csv_dict = {}

for key, hl in headers_dict.items():
    header_string = ''
    header_string += hl[0]
    for i in range(1,len(hl)):
        header_string += ';'
        header_string += hl[i]
    header_string += '\n'    
    headers_csv_dict[key] = header_string

#headers_csv_dict

In [ ]:
# create a string for each entry (row) or the 3 datasets - csv files

#initialize the 3 dataset lists
conversation_list = []
messages_list = []
reactions_list = []

#initialize the 3 keys
key_c = 1
key_m = 1
key_r = 1

for root, dirs, files in os.walk(path_to_source_folder):
    for file in files:
        if file[:7] == 'message':
            path = root+'/'+file
            with open(path, encoding = 'utf8') as file:
                conversation_dict = json.load(file)
                
                # create the messages dataset row 
                #iterate through the internal messages list
                for message in conversation_dict['messages']:
                    message_sender_name = mojibake(message['sender_name'])
                    message_timestamp = message['timestamp_ms']
                    message_text = ''
                    if 'content' in message.keys(): message_text = mojibake(message['content']).replace(';','?').replace('\n', ' -new line- ')
                    
                    message_type = ''
#                     if 
#                     elif
#                     elif
                                        
                    message_call_duration = ''
                    if 'call_duration' in message.keys(): message_call_duration = message['call_duration']
                    message_link = ''
                    if 'share' in message.keys():
                        if 'link' in message['share'].keys():
                            message_link = message['share']['link'].replace(';','?')
                    message_is_unsent = False
                    if 'is_unsent' in message.keys(): message_is_unsent = message['is_unsent']  
                    
                    temp_m = f'{key_m};{key_c};{message_sender_name};{message_timestamp};{message_text};{message_type};{message_call_duration};{message_link};{message_is_unsent}\n'
                    messages_list.append(temp_m)
                    
                    # create the reactions dataset row     
                    if 'reactions' in message.keys():
                        for reaction in message['reactions']:
                            reaction_reaction = mojibake(reaction['reaction'])
                            reaction_actor = mojibake(reaction['actor'])
                            
                            temp_r = f'{key_r};{key_m};{reaction_reaction};{reaction_actor}\n'
                            reactions_list.append(temp_r)
                            
                            # increase the reaction counter key
                            key_r += 1  
                    
                    # increase the messages counter key
                    key_m += 1
                    
    
            # create the conversation dataset row 
            # if many "message_x.json" files exist, the entry will be executed only for the last one
            # all files are part of the same conversation and store the same data
            conversation_title = mojibake(conversation_dict['title'])
            conversation_no_of_participants = len(conversation_dict['participants'])
            conversation_is_still_participant = conversation_dict['is_still_participant']
            conversation_name_of_participants = ''
            for name in conversation_dict['participants']:
                conversation_name_of_participants += mojibake(name['name'])
                conversation_name_of_participants += ','
            conversation_name_of_participants = conversation_name_of_participants[:-1] #cut the last semicolon 
            temp_c = f'{key_c};{conversation_name_of_participants};{conversation_title};{conversation_is_still_participant};{conversation_no_of_participants}\n'
            conversation_list.append(temp_c)                    
            # increase the conversation counter key
            key_c += 1

In [ ]:
# print(len(conversation_list))
# conversation_list[:10]

In [ ]:
# print(len(messages_list))
# messages_list[:10]

In [ ]:
# print(len(reactions_list))
# reactions_list[:10]

In [ ]:
#create the output csv files
data_dict = {
    'conversations' : conversation_list,
    'messages' : messages_list,
    'reactions' : reactions_list
}

date_time = dt.now().strftime("%Y.%m.%d_%H.%M")  # '2023.08.16_15.39'

for name, value in headers_csv_dict.items():
    # headers
    file_name = name+'_'+date_time+'.csv'
    with open(file_name, 'w', encoding='UTF8') as output:
        output.write(value)
    # rows 
    #iterate through the list and add all entries to the file    
    with open(file_name, 'a', encoding='UTF8') as output:
        for i in data_dict[name]:
            output.write(i)